In [69]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import seaborn as sns
import matplotlib .pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn .svm import SVC
import joblib
from joblib import dump
import h5py
import json

In [70]:
file_path = 'D:\\Dataset\\#4_전력 설비 에너지 패턴 및 고장 분석 센서\\Training\\1.펌프_일반모터\\2.SOH진단\\1.역률평균\\0.Combined\\'
files = ['Combined_LabelledData_346_역률평균.json', 'Combined_LabelledData_348_역률평균.json']
x_label = []
y_label = []

for file_name in files:
    full_path = file_path + file_name
    with open(full_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for entry in data['data']:
        if entry['ITEM_NAME'] == "역률평균":
            x_label.append(entry['ITEM_VALUE'])
            y_label.append(entry['LABEL_NAME'])

x_label = np.array(x_label)
x_label_scaled = (x_label - np.mean(x_label)) / np.std(x_label)


In [71]:
data_indices = np.arange(len(x_label))
print(data_indices)
np.random.shuffle(data_indices)
print(data_indices)

x_label = np.array(x_label)[data_indices]
y_label = np.array(y_label)[data_indices]

x_train, x_temp, y_train, y_temp = train_test_split(x_label, y_label, test_size=0.4, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)

print(f"Train size: {len(x_train)}")
print(f"Validation size: {len(x_valid)}")
print(f"Test size: {len(x_test)}")


[    0     1     2 ... 57530 57531 57532]
[ 8738  8505 43120 ... 24303 12468 19775]
Train size: 34519
Validation size: 11507
Test size: 11507


In [74]:
def train_and_save_model(x_train, y_train, model_save_path):
    # RandomForestClassifier 모델 초기화 및 학습
    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # 모델 저장
    joblib.dump(model, model_save_path)

    # 테스트 데이터로 예측
    y_pred = model.predict(x_test)

    # 정확도 및 분류 보고서 출력
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.4f}')
    
def model_to_h5(model_save_path, h5_save_path):
    # 모델 불러오기
    model = joblib.load(model_save_path)

    # H5 파일로 저장
    with h5py.File(h5_save_path, 'w') as hf:
        for key, value in model.__dict__.items():
            try:
                hf.create_dataset(key, data=value)
            except TypeError:
                pass

In [75]:
model_save_path = 'C:\\Users\\user\\Desktop\\test\\Define\\Define_add\\rf_model.joblib'
h5_save_path = 'C:\\Users\\user\\Desktop\\test\\Define\\Define_add\\rf_model.h5'
train_and_save_model(x_train, y_train, model_save_path)
model_to_h5(model_save_path, h5_save_path)

Accuracy: 0.9966
